<a href="https://colab.research.google.com/github/cod3astro/ML_practice/blob/main/codon_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [2]:
df = pd.read_csv('codon_usage.csv', index_col='SpeciesID')
df.drop_duplicates(inplace=True)
df.head()

/tmp/ipython-input-2-2076267449.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('codon_usage.csv', index_col='SpeciesID')


,Kingdom,DNAtype,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
SpeciesID,,,,,,,,,,,,,,,,,,,,,
100217,vrl,0,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
100220,vrl,0,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
100755,vrl,0,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
100880,vrl,0,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
100887,vrl,0,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [3]:
df.shape

(13028, 68)

In [4]:
df.columns

Index(['Kingdom', 'DNAtype', 'Ncodons', 'SpeciesName', 'UUU', 'UUC', 'UUA',
       'UUG', 'CUU', 'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUU',
       'GUC', 'GUA', 'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU', 'CCC', 'CCA',
       'CCG', 'UGG', 'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC', 'UCA', 'UCG',
       'AGU', 'AGC', 'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC', 'CAA', 'CAG',
       'AAU', 'AAC', 'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG', 'CGU', 'CGC',
       'CGA', 'CGG', 'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG', 'UAA', 'UAG',
       'UGA'],
      dtype='object')

In [5]:
df.isnull().any().sum()

np.int64(0)

In [6]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

In [7]:
features = ['DNAtype', 'UUA',
       'UUG', 'CUU', 'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUU',
       'GUC', 'GUA', 'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU', 'CCC', 'CCA',
       'CCG', 'UGG', 'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC', 'UCA', 'UCG',
       'AGU', 'AGC', 'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC', 'CAA', 'CAG',
       'AAU', 'AAC', 'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG', 'CGU', 'CGC',
       'CGA', 'CGG', 'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG', 'UAA', 'UAG',
       'UGA']
mi_features = ['DNAtype', 'Ncodons', 'UUU', 'UUC']
target = 'Kingdom'
X = df[features]
y = df[target]
smote = SMOTE(random_state=42)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42, stratify=y, test_size=0.2)

In [8]:
Features = ['DNAtype', 'Ncodons', 'SpeciesName', 'UUU', 'UUC', 'UUA',
       'UUG', 'CUU', 'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUU',
       'GUC', 'GUA', 'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU', 'CCC', 'CCA',
       'CCG', 'UGG', 'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC', 'UCA', 'UCG',
       'AGU', 'AGC', 'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC', 'CAA', 'CAG',
       'AAU', 'AAC', 'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG', 'CGU', 'CGC',
       'CGA', 'CGG', 'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG', 'UAA', 'UAG',
       'UGA']
Target = 'Kingdom'
a = df[Features]
b = df[Target]
train_a, val_a, train_b, val_b = train_test_split(a, b, random_state=42, stratify=b, test_size=0.2)

In [9]:
for col in X:
  if X[col].dtype == 'object':
    print(col)

In [10]:
for col in X.columns:
    if X[col].dtype == 'object':
        # Find rows with non-numeric values
        non_numeric = X[col][~X[col].apply(lambda x: str(x).replace(';', '').isdigit())]
        if not non_numeric.empty:
            print(f"Column '{col}' has non-numeric values:")
            print(non_numeric.unique())

In [11]:
for col in X.columns:
    # Check if any value is a string that can't convert to float
    non_float = []
    for val in X[col]:
        try:
            float(val)
        except (ValueError, TypeError):
            non_float.append(val)

    if non_float:
        print(f"Column '{col}' has non-float values:")
        print(pd.Series(non_float).unique())

In [12]:
print("train_X dtypes:", train_X.dtypes)
print("val_X dtypes:", val_X.dtypes)

train_X dtypes: DNAtype      int64
UUA        float64
UUG        float64
CUU        float64
CUC        float64
            ...   
GAA        float64
GAG        float64
UAA        float64
UAG        float64
UGA        float64
Length: 63, dtype: object
val_X dtypes: DNAtype      int64
UUA        float64
UUG        float64
CUU        float64
CUC        float64
            ...   
GAA        float64
GAG        float64
UAA        float64
UAG        float64
UGA        float64
Length: 63, dtype: object


In [13]:
copy_X = X.copy()
copy_y = y.copy()
for col in copy_X.select_dtypes('float'):
    copy_X[col] = copy_X[col].round().astype(int)
for char in copy_X.select_dtypes('object'):
  copy_X.loc[:, char], _ = copy_X[char].factorize()
discrete_values = copy_X.dtypes == int
copy_y, _ = pd.factorize(y)

def make_mi_score(copy_X, copy_y, discrete_values):
    mi_scores = mutual_info_classif(copy_X, copy_y, discrete_features=discrete_values)
    mi_scores = pd.Series(mi_scores, name='MI SCORES', index=copy_X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_score(copy_X, copy_y, discrete_values)
mi_scores[:7]

,MI SCORES
DNAtype,0.403521
UUA,0.000000
UUG,0.000000
CUU,0.000000
CUC,0.000000
CUA,0.000000
CUG,0.000000


In [14]:
y.value_counts(normalize=True)

,proportion
Kingdom,
bct,0.224133
vrl,0.217378
pln,0.193660
vrt,0.159426
inv,0.103239
mam,0.043905
phg,0.016887
rod,0.016503
pri,0.013816


In [15]:
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, random_state=42, stratify=y_resampled, test_size=0.2)

In [26]:
model = Pipeline([
    ('model', XGBClassifier(random_state=42, eval_metric='mlogloss', n_estimators=300, max_depth=6, learning_rate=0.14))
])
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

model.fit(train_X, train_y_encoded)
xgb_train_preds_encoded = model.predict(val_X)
xgb_train_preds = label_encoder.inverse_transform(xgb_train_preds_encoded)
accuracy_score(val_y, xgb_train_preds)

0.930928626247122

In [17]:
model = Pipeline([
    ('model', XGBClassifier(random_state=42, eval_metric='mlogloss', n_estimators=300, max_depth=7, learning_rate=0.13))
])
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

model.fit(X_train, y_train_encoded)
xgb_train_preds_encoded = model.predict(X_val)
xgb_train_preds = label_encoder.inverse_transform(xgb_train_preds_encoded)
print("Accuracy:", accuracy_score(y_val, xgb_train_preds))
print("Precision:", precision_score(y_val, xgb_train_preds, average='macro'))
print("Recall:", recall_score(y_val, xgb_train_preds, average='macro'))
print("F1 Score:", f1_score(y_val, xgb_train_preds, average='macro'))
print("Confusion Matrix:\n", confusion_matrix(y_val, xgb_train_preds))

Accuracy: 0.9834993773349938
Precision: 0.9835438942196021
Recall: 0.9834993773349939
F1 Score: 0.9834660372409999
Confusion Matrix:
 [[584   0   0   0   0   0   0   0   0   0   0]
 [  7 553   5   0  11   1   4   0   0   3   0]
 [  0   1 566   0   0   0   9   0   0   5   3]
 [  0   0   0 583   0   0   0   1   0   0   0]
 [  0   0   0   0 584   0   0   0   0   0   0]
 [  0   0   0   0   0 584   0   0   0   0   0]
 [  0   7   8   0   0   0 562   0   0   7   0]
 [  0   0   0   0   0   0   0 583   1   0   0]
 [  0   0   1   1   0   0   0   0 582   0   0]
 [  1   1   8   0   1   0   7   0   0 566   0]
 [  0   1   5   2   0   0   0   2   0   3 571]]


In [18]:
model = Pipeline([
    ('model', RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=300, max_depth=30))
])
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

model.fit(train_X, train_y_encoded)
rf_train_preds_encoded = model.predict(val_X)
rf_train_preds = label_encoder.inverse_transform(rf_train_preds_encoded)
accuracy_score(val_y, rf_train_preds)

0.9071373752877974

In [19]:
model = Pipeline([
    ('model', RandomForestClassifier(random_state=42, n_estimators=300, max_depth=20))
])
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

model.fit(X_train, y_train_encoded)
rf_train_preds_encoded = model.predict(X_val)
rf_train_preds = label_encoder.inverse_transform(rf_train_preds_encoded)
accuracy_score(y_val, rf_train_preds)

0.9796077210460772

In [20]:
scores = cross_val_score(
    RandomForestClassifier(),
    X[['DNAtype']].apply(lambda x: pd.factorize(x)[0]),  # Convert to numeric
    y,
    cv=5
)
print(f"Accuracy using only DNAtype: {scores.mean():.2f}")

Accuracy using only DNAtype: 0.41


In [30]:
class TargetEncoderWrapper:
    def __init__(self, pipeline, encoder):
        self.pipeline = pipeline
        self.encoder = encoder

    def predict(self, X):
        encoded = self.pipeline.predict(X)
        return self.encoder.inverse_transform(encoded)

def validate_data(X, numeric_features, one_hot_features, ordinal_features):
    #Enhanced validation that considers dynamic features
    errors = []

    # Check for null values in specified features
    all_features = numeric_features + one_hot_features + ordinal_features
    if all_features:  # Only check if features are specified
        null_cols = X[all_features].isnull().any()
        if null_cols.any():
            errors.append(f"Null values found in columns: {null_cols[null_cols].index.tolist()}")

    # Check for empty strings in categorical features
    cat_features = one_hot_features + ordinal_features
    if cat_features:
        empty_str_cols = [
            col for col in cat_features
            if col in X.columns and X[col].astype(str).str.strip().eq('').any()
        ]
        if empty_str_cols:
            errors.append(f"Empty strings found in columns: {empty_str_cols}")

    # Check numeric features contain valid values
    for col in numeric_features:
        if col in X.columns:
            if not pd.api.types.is_numeric_dtype(X[col]):
                errors.append(f"Non-numeric values found in numeric column: {col}")

    if errors:
        raise ValueError("Validation failed:\n- " + "\n- ".join(errors))

    return True

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop=None, rename_columns=None, index_col=None,
                 numeric_features=None, ordinal_features=None, one_hot_features=None):
        self.columns_to_drop = columns_to_drop or []
        self.rename_columns = rename_columns or {}
        self.index_col = index_col
        self.numeric_features = numeric_features or []
        self.ordinal_features = ordinal_features or []
        self.one_hot_features = one_hot_features or []

    def fit(self, X, y=None):
        # Only calculate statistics for features that exist
        if self.numeric_features:
            existing_num = [col for col in self.numeric_features if col in X.columns]
            self.num_means_ = X[existing_num].mean() if existing_num else pd.Series()

        cat_features = self.one_hot_features + self.ordinal_features
        if cat_features:
            existing_cat = [col for col in cat_features if col in X.columns]
            self.cat_modes_ = X[existing_cat].mode().iloc[0] if existing_cat else pd.Series()

        return self

    def transform(self, X):
        X_clean = X.copy()

        # Validate before any transformations
        validate_data(
            X_clean,
            numeric_features=self.numeric_features,
            one_hot_features=self.one_hot_features,
            ordinal_features=self.ordinal_features
        )

        # Set index if specified
        if self.index_col and self.index_col in X_clean.columns:
            X_clean = X_clean.set_index(self.index_col)

        # Drop columns
        existing_drops = [col for col in self.columns_to_drop if col in X_clean.columns]
        X_clean = X_clean.drop(columns=existing_drops, errors='ignore')

        # Rename columns
        existing_renames = {old: new for old, new in self.rename_columns.items()
                          if old in X_clean.columns}
        X_clean = X_clean.rename(columns=existing_renames)

        # Fill NA values
        if hasattr(self, 'num_means_') and not self.num_means_.empty:
            num_cols = [col for col in self.numeric_features
                       if col in X_clean.columns and col in self.num_means_.index]
            X_clean[num_cols] = X_clean[num_cols].fillna(self.num_means_)

        if hasattr(self, 'cat_modes_') and not self.cat_modes_.empty:
            cat_cols = [col for col in (self.one_hot_features + self.ordinal_features)
                       if col in X_clean.columns and col in self.cat_modes_.index]
            X_clean[cat_cols] = X_clean[cat_cols].fillna(self.cat_modes_)

        # Clean strings
        str_cols = X_clean.select_dtypes(include=['object']).columns
        for col in str_cols:
            X_clean[col] = X_clean[col].astype(str).str.strip().str.lower()

        return X_clean.drop_duplicates()

def build_pipeline(numeric_features=None, one_hot_features=None, ordinal_features=None,
                  columns_to_drop=None, rename_columns=None, index_col=None):

    # Initialize feature lists
    numeric_features = numeric_features or []
    one_hot_features = one_hot_features or []
    ordinal_features = ordinal_features or []

    # Build transformers
    transformers = []
    if numeric_features:
        transformers.append(('num', Pipeline([('scaler', StandardScaler())]), numeric_features))
    if one_hot_features:
        transformers.append(('cat', Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))]), one_hot_features))
    if ordinal_features:
        transformers.append(('ord', Pipeline([('ordinal', OrdinalEncoder())]), ordinal_features))

    preprocessor = ColumnTransformer(transformers=transformers) if transformers else None

    steps = [
        ('cleaner', DataCleaner(
            columns_to_drop=columns_to_drop,
            rename_columns=rename_columns,
            index_col=index_col,
            numeric_features=numeric_features,
            one_hot_features=one_hot_features,
            ordinal_features=ordinal_features
        ))
    ]

    if preprocessor:
        steps.append(('preprocessor', preprocessor))

    steps.append(('classifier', XGBClassifier(
        random_state=42,
        n_estimators=300,
        max_depth=7,
        learning_rate=0.13,
        use_label_encoder=False,
        eval_metric='mlogloss'
    )))

    return Pipeline(steps)

def train_and_save_model(X_train, y_train, save_path='model.joblib', **pipeline_args):
    # Encode target
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_train)

    # Build and train pipeline
    pipeline = build_pipeline(**pipeline_args)
    pipeline.fit(X_train, y_encoded)

    # Save both components
    joblib.dump({
        'pipeline': pipeline,
        'label_encoder': label_encoder
    }, save_path)

    return pipeline, label_encoder

def load_and_predict(model_path, new_data):
    # Load components
    model_data = joblib.load(model_path)
    pipeline = model_data['pipeline']
    label_encoder = model_data['label_encoder']

    # Make predictions
    encoded_preds = pipeline.predict(new_data)
    return label_encoder.inverse_transform(encoded_preds)

In [22]:
print(f"Train features shape: {train_a.shape}")
print(f"Train labels shape: {train_b.shape}")

Train features shape: (10422, 67)
Train labels shape: (10422,)


In [33]:
# Define the features after dropping the specified columns
all_features = [col for col in df.columns if col not in ['Kingdom', 'SpeciesID', 'SpeciesName', 'UUU', 'UUC', 'Ncodons']]


# 1. Build and train pipeline
pipeline = build_pipeline(
    numeric_features=[],
    one_hot_features=[],
    ordinal_features=[],
    columns_to_drop=['SpeciesName', 'UUU', 'UUC', 'Ncodons'],
    rename_columns={},
    index_col='SpeciesID'
)

# Apply the cleaner to both training features and target
cleaner = pipeline.named_steps['cleaner']

print("Shape of train_a before cleaning:", train_a.shape)
print("Shape of train_b before cleaning:", train_b.shape)

train_a_cleaned, train_b_cleaned = cleaner.transform(train_a, train_b)

print("Shape of train_a after cleaning:", train_a_cleaned.shape)
print("Shape of train_b after cleaning:", train_b_cleaned.shape)

# Encode the cleaned target variable
label_encoder = LabelEncoder()
train_b_encoded = label_encoder.fit_transform(train_b_cleaned)

# Train with encoded target
pipeline.fit(train_a_cleaned, train_b_encoded)

# 3. Save both pipeline and encoder
joblib.dump({
    'pipeline': pipeline,
    'label_encoder': label_encoder
}, 'my_pipeline.joblib')

Shape of train_a before cleaning: (10422, 67)
Shape of train_b before cleaning: (10422,)


TypeError: DataCleaner.transform() takes 2 positional arguments but 3 were given